# **Install and import libraries**

In [1]:
pip install selenium

     |████████████████████████████████| 911kB 2.8MB/s 


In [0]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

In [5]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

E: dpkg was interrupted, you must manually run 'dpkg --configure -a' to correct the problem. 
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


# **Get data from wikipedia using the link**

In [0]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [0]:
content = wd.page_source
soup = BeautifulSoup(content)

# **Get the table of neibourhoods in Toronto from wikipedia**

In [0]:
table = soup.find('table', attrs={'class':'wikitable sortable jquery-tablesorter'})
df = pd.read_html(str(table))[0]

In [94]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# **Ignore cells with a borough that is Not assigned**

In [0]:
df = df[df['Borough']!='Not assigned']

In [111]:
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


In [0]:
df = df.reset_index(drop=True)

# **Combine rows with the same PostCode in one row by combining neighbourhoods**

In [166]:
df2 = pd.DataFrame(columns = ('Postcode','Borough','Neighbourhood'))
df2

,Postcode,Borough,Neighbourhood


In [0]:
ind = 0
for i in range(len(df)):
  if (df['Postcode'][i] in df2.Postcode.values) == True :
    for j in range(len(df2)):
      if df2['Postcode'][j] == df['Postcode'][i]:
        df2['Neighbourhood'][j] = (df2['Neighbourhood'][j] + ', ' +df['Neighbourhood'][i])
  else :
    df2.loc[ind] = list(df.loc[i])
    ind = ind+1

# **Display dataframe**

In [168]:
df2

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


# **Print the number of rows in the finale dataframe**

In [175]:
print('The number of rows in our finale dataframe is : ' + str(df2.shape[0]))

The number of rows in our finale dataframe is : 103


# **Download the dataframe as a csv file**

In [0]:
df2.to_csv('data.csv')